# **Regression**
regression tree-based model

In [1]:
import numpy as np
import pandas as pd

In [14]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score, precision_score, recall_score, confusion_matrix

In [2]:
df = pd.read_csv('/content/df_CPs.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19981 entries, 0 to 19980
Data columns (total 62 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  19981 non-null  int64  
 1   PC1         19981 non-null  float64
 2   PC2         19981 non-null  float64
 3   PC3         19981 non-null  float64
 4   PC4         19981 non-null  float64
 5   PC5         19981 non-null  float64
 6   PC6         19981 non-null  float64
 7   PC7         19981 non-null  float64
 8   PC8         19981 non-null  float64
 9   PC9         19981 non-null  float64
 10  PC10        19981 non-null  float64
 11  PC11        19981 non-null  float64
 12  PC12        19981 non-null  float64
 13  PC13        19981 non-null  float64
 14  PC14        19981 non-null  float64
 15  PC15        19981 non-null  float64
 16  PC16        19981 non-null  float64
 17  PC17        19981 non-null  float64
 18  PC18        19981 non-null  float64
 19  PC19        19981 non-nul

In [19]:
df = df.drop(columns=['smiles'])

In [20]:
X = df.iloc[:, :60]  # Features
y = df['gap']  # целевая переменная

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1984)

In [22]:
regressor = DecisionTreeRegressor(random_state=1984)
regressor.fit(X_train, y_train)
y_pred = regressor.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

Mean Squared Error: 0.004350603398447283
